In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['DA']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('dopamine', 10859),
 ('dopaminergic', 1173),
 ('ductus arteriosus', 119),
 ('domoic acid', 87),
 ('dark agouti', 49),
 ('dopamine agonist', 44),
 ('dorsal aorta', 35),
 ('dafachronic acid', 29),
 ('decursinol angelate', 26),
 ('drugs abuse', 23),
 ('danaparoid sodium', 18),
 ('diminazene aceturate', 17),
 ('darbepoetin alfa', 16),
 ('dopaminergic neurons', 16),
 ('diels alder', 13),
 ('diacetyl', 13),
 ('distal arthrogryposis', 11),
 ('axial diffusivity', 11),
 ('destruxin a', 10),
 ('dehydroandrographolide', 10),
 ('dark adaptations', 9),
 ('discriminant analysis', 9),
 ('dehydroabietinal', 9),
 ('deoxycholic acid', 9),
 ('decanoic acid', 9),
 ('dasatinib', 8),
 ('divided attention', 8),
 ('degree of anisotropy', 8),
 ('dopa mine', 8),
 ('darbepoetin alpha', 7),
 ('degree of acetylation', 7),
 ('decision aid', 6),
 ('dendritic arborization', 6),
 ('direct asymmetry', 6),
 ('dominant activity', 6),
 ('diamide', 6),
 ('daniels', 5),
 ('desmodium adscendens', 5),
 ('dense area', 5),
 (

In [8]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [9]:
names

{'CHEBI:CHEBI:18243': 'dopamine',
 'MESH:D004373': 'Ductus Arteriosus',
 'CHEBI:CHEBI:34727': 'domoic acid',
 'EFO:0005291': 'Dark agouti',
 'CHEBI:CHEBI:51065': 'dopamine agonist',
 'MESH:D001011': 'Aorta',
 'MESH:C533800': 'dafachronic acid',
 'CHEBI:CHEBI:52634': 'sodium-23 atom',
 'MESH:C003915': 'diminazene aceturate',
 'HGNC:4392': 'GNAS',
 'MESH:D059290': 'Dopaminergic Neurons',
 'MESH:D029661': 'Alnus',
 'CHEBI:CHEBI:16583': 'butane-2,3-dione',
 'HP:HP:0005684': 'Distal arthrogryposis',
 'MESH:C040423': 'destruxin A',
 'MESH:C478098': 'dehydroandrographolide',
 'MESH:D016002': 'Discriminant Analysis',
 'MESH:C576115': 'dehydroabietinal',
 'CHEBI:CHEBI:28834': 'deoxycholic acid',
 'CHEBI:CHEBI:30813': 'decanoic acid',
 'CHEBI:CHEBI:49375': 'dasatinib (anhydrous)',
 'MESH:D001288': 'Attention',
 'MESH:D016880': 'Anisotropy',
 'CHEBI:CHEBI:30216': 'alpha-particle',
 'MESH:D000107': 'Acetylation',
 'HGNC:13203': 'AICDA',
 'CHEBI:CHEBI:48958': "1,1'-azobis(N,N-dimethylformamide)",
 

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-04 01:59:57] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'axial diffusivity': 'ungrounded',
  'da': 'ungrounded',
  'dafachronic acid': 'MESH:C533800',
  'daidzein': 'CHEBI:CHEBI:28197',
  'danaparoid sodium': 'ungrounded',
  'daniels': 'ungrounded',
  'darbepoetin alfa': 'ungrounded',
  'darbepoetin alpha': 'ungrounded',
  'darbepoetin α': 'ungrounded',
  'dark adaptations': 'ungrounded',
  'dark agouti': 'EFO:EFO:0005291',
  'dasatinib': 'CHEBI:CHEBI:49375',
  'decanoic': 'CHEBI:CHEBI:30813',
  'decanoic acid': 'CHEBI:CHEBI:30813',
  'decision aid': 'ungrounded',
  'decursinol angelate': 'PUBCHEM:776123',
  'deformation amplitude': 'ungrounded',
  'degree of acetylation': 'MESH:D000107',
  'degree of anisotropy': 'MESH:D016880',
  'dehydroabietic acid': 'CHEBI:CHEBI:29571',
  'dehydroabietinal': 'MESH:C576115',
  'dehydroandrographolide': 'MESH:C478098',
  'dehydromatricarin a': 'ungrounded',
  'delayed assessed': 'ungrounded',
  'dendritic arborization': 'ungrounded',
  'dense area': 'ungrounded',
  'dental adhesive': 'ungrounded',
  'd

In [13]:
grounding_map, names, pos_labels = [{'axial diffusivity': 'ungrounded',
  'da': 'ungrounded',
  'dafachronic acid': 'MESH:C533800',
  'daidzein': 'CHEBI:CHEBI:28197',
  'danaparoid sodium': 'ungrounded',
  'daniels': 'ungrounded',
  'darbepoetin alfa': 'ungrounded',
  'darbepoetin alpha': 'ungrounded',
  'darbepoetin α': 'ungrounded',
  'dark adaptations': 'ungrounded',
  'dark agouti': 'EFO:EFO:0005291',
  'dasatinib': 'CHEBI:CHEBI:49375',
  'decanoic': 'CHEBI:CHEBI:30813',
  'decanoic acid': 'CHEBI:CHEBI:30813',
  'decision aid': 'ungrounded',
  'decursinol angelate': 'PUBCHEM:776123',
  'deformation amplitude': 'ungrounded',
  'degree of acetylation': 'MESH:D000107',
  'degree of anisotropy': 'MESH:D016880',
  'dehydroabietic acid': 'CHEBI:CHEBI:29571',
  'dehydroabietinal': 'MESH:C576115',
  'dehydroandrographolide': 'MESH:C478098',
  'dehydromatricarin a': 'ungrounded',
  'delayed assessed': 'ungrounded',
  'dendritic arborization': 'ungrounded',
  'dense area': 'ungrounded',
  'dental adhesive': 'ungrounded',
  'deoxyandrographolide': 'ungrounded',
  'deoxycholic acid': 'CHEBI:CHEBI:28834',
  'descending aorta': 'ungrounded',
  'desmodium adscendens': 'ungrounded',
  'destruxin a': 'MESH:C040423',
  'developmental age': 'ungrounded',
  'dexamethasone acetate': 'CHEBI:CHEBI:4463',
  'diaceturate': 'ungrounded',
  'diacetyl': 'CHEBI:CHEBI:16583',
  'diamide': 'CHEBI:CHEBI:48958',
  'diazirine': 'CHEBI:CHEBI:51624',
  'dicarboxylic acid': 'CHEBI:CHEBI:35692',
  'diels alder': 'ungrounded',
  'differentiation abundant': 'ungrounded',
  'diisocyanate asthma': 'ungrounded',
  'dilute acid': 'ungrounded',
  'dimethylacrylshikonin': 'ungrounded',
  'diminazene aceturate': 'CHEBI:CHEBI:82615',
  'dioscorea alata': 'ungrounded',
  'direct access': 'CHEBI:CHEBI:34922',
  'direct activity': 'ungrounded',
  'direct asymmetry': 'ungrounded',
  'discordant alternans': 'ungrounded',
  'discriminant analysis': 'MESH:D016002',
  'displacement adjustment': 'ungrounded',
  'distal appendages': 'ungrounded',
  'distal arthrogryposis': 'HP:HP:0005684',
  'divided attention': 'ungrounded',
  'dodecanoic acid': 'CHEBI:CHEBI:30805',
  'dominant activity': 'ungrounded',
  'domoic acid': 'CHEBI:CHEBI:34727',
  'donor acceptor': 'ungrounded',
  'dopa mine': 'CHEBI:CHEBI:18243',
  'dopaminc': 'CHEBI:CHEBI:18243',
  'dopamine': 'CHEBI:CHEBI:18243',
  'dopamine agonist': 'CHEBI:CHEBI:51065',
  'dopamine hcl': 'CHEBI:CHEBI:18243',
  'dopamine release': 'ungrounded',
  'dopamine s': 'CHEBI:CHEBI:18243',
  'dopaminergic': 'CHEBI:CHEBI:18243',
  'dopaminergic agonist': 'CHEBI:CHEBI:51065',
  'dopaminergic neurons': 'MESH:D059290',
  'dopamlne': 'CHEBI:CHEBI:18243',
  'dorsal aorta': 'MESH:D001011',
  'dorsal appendages': 'ungrounded',
  'dorsal attention': 'ungrounded',
  'drugs abuse': 'ungrounded',
  'ductus arteriosus': 'MESH:D004373',
  'dynamics assessed': 'ungrounded'},
 {'MESH:C533800': 'dafachronic acid',
  'CHEBI:CHEBI:28197': 'daidzein',
  'EFO:EFO:0005291': 'Dark agouti',
  'CHEBI:CHEBI:49375': 'dasatinib (anhydrous)',
  'CHEBI:CHEBI:30813': 'decanoic acid',
  'PUBCHEM:776123': 'Decursinol angelate',
  'MESH:D000107': 'Acetylation',
  'MESH:D016880': 'Anisotropy',
  'CHEBI:CHEBI:29571': 'dehydroabietic acid',
  'MESH:C576115': 'dehydroabietinal',
  'MESH:C478098': 'dehydroandrographolide',
  'CHEBI:CHEBI:28834': 'deoxycholic acid',
  'MESH:C040423': 'destruxin A',
  'CHEBI:CHEBI:4463': 'Dexamethasone acetate',
  'CHEBI:CHEBI:16583': 'butane-2,3-dione',
  'CHEBI:CHEBI:48958': "1,1'-azobis(N,N-dimethylformamide)",
  'CHEBI:CHEBI:51624': 'diazirine',
  'CHEBI:CHEBI:35692': 'dicarboxylic acid',
  'CHEBI:CHEBI:82615': 'diminazene diaceturate',
  'CHEBI:CHEBI:34922': 'picloram',
  'MESH:D016002': 'Discriminant Analysis',
  'HP:HP:0005684': 'Distal arthrogryposis',
  'CHEBI:CHEBI:30805': 'dodecanoic acid',
  'CHEBI:CHEBI:34727': 'domoic acid',
  'CHEBI:CHEBI:18243': 'dopamine',
  'CHEBI:CHEBI:51065': 'dopamine agonist',
  'MESH:D059290': 'Dopaminergic Neurons',
  'MESH:D001011': 'Aorta',
  'MESH:D004373': 'Ductus Arteriosus'},
 ['CHEBI:CHEBI:18243', 'CHEBI:CHEBI:34727', 'MESH:D004373']]

In [14]:
excluded_longforms = ['da']

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-04 02:30:15] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-04 02:36:52] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9860805775970312 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:18243': 9488,
  'MESH:D004373': 92,
  'CHEBI:CHEBI:28834': 6,
  'EFO:EFO:0005291': 30,
  'CHEBI:CHEBI:35692': 4,
  'CHEBI:CHEBI:16583': 7,
  'ungrounded': 125,
  'CHEBI:CHEBI:4463': 2,
  'CHEBI:CHEBI:48958': 4,
  'CHEBI:CHEBI:51065': 41,
  'CHEBI:CHEBI:34727': 56,
  'MESH:D016002': 6,
  'MESH:D001011': 23,
  'CHEBI:CHEBI:30813': 13,
  'MESH:D059290': 15,
  'CHEBI:CHEBI:82615': 12,
  'HP:HP:0005684': 7,
  'PUBCHEM:776123': 12,
  'MESH:C533800': 20,
  'MESH:C040423': 6,
  'MESH:C576115': 5,
  'CHEBI:CHEBI:28197': 3,
  'MESH:D016880': 6,
  'CHEBI:CHEBI:49375': 4,
  'MESH:D000107': 5,
  'CHEBI:CHEBI:29571': 3,
  'CHEBI:CHEBI:34922': 2,
  'MESH:C478098': 6,
  'CHEBI:CHEBI:51624': 2,
  'CHEBI:CHEBI:30805': 1},
 'f1': {'mean': 0.986081, 'std': 0.001537},
 'precision': {'mean': 0.9779, 'std': 0.001832},
 'recall': {'mean': 0.995122, 'std': 0.001845},
 'CHEBI:CHEBI:82615': {'f1': {'mean': 0.233333, 'std': 0.290593},
  'pr': {'mean': 0.2, 'std': 0.266667},
  

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [23]:
print(disamb.info())

Disambiguation model for DA

Produces the disambiguations:
	1,1'-azobis(N,N-dimethylformamide)	CHEBI:CHEBI:48958
	Acetylation	MESH:D000107
	Anisotropy	MESH:D016880
	Aorta	MESH:D001011
	Dark agouti	EFO:EFO:0005291
	Decursinol angelate	PUBCHEM:776123
	Dexamethasone acetate	CHEBI:CHEBI:4463
	Discriminant Analysis	MESH:D016002
	Distal arthrogryposis	HP:HP:0005684
	Dopaminergic Neurons	MESH:D059290
	Ductus Arteriosus*	MESH:D004373
	butane-2,3-dione	CHEBI:CHEBI:16583
	dafachronic acid	MESH:C533800
	daidzein	CHEBI:CHEBI:28197
	dasatinib (anhydrous)	CHEBI:CHEBI:49375
	decanoic acid	CHEBI:CHEBI:30813
	dehydroabietic acid	CHEBI:CHEBI:29571
	dehydroabietinal	MESH:C576115
	dehydroandrographolide	MESH:C478098
	deoxycholic acid	CHEBI:CHEBI:28834
	destruxin A	MESH:C040423
	diazirine	CHEBI:CHEBI:51624
	dicarboxylic acid	CHEBI:CHEBI:35692
	diminazene diaceturate	CHEBI:CHEBI:82615
	dodecanoic acid	CHEBI:CHEBI:30805
	domoic acid*	CHEBI:CHEBI:34727
	dopamine*	CHEBI:CHEBI:18243
	dopamine agonist	CHEBI:CHEB

In [24]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1